In [ ]:
import pandas as pd 
import os
import os 
import argparse
import subprocess
import pandas as pd

In [ ]:
def retrive_ref(tax,taxid=False):
    print("\033[;32;1mSearching...\033[;39;m")
    if taxid == False:
        out = subprocess.Popen(f"esearch -db assembly -query '{tax}[ORGN]' | esummary | xtract -pattern DocumentSummary \
                -def NA -element Organism,AssemblyAccession,Taxid,assembly-status -block Stat \
                -if Stat@category -equals contig_count -or Stat@category -equals contig_l50 \
                -or Stat@category -equals contig_n50 -or Stat@category -equals total_length \
                -element Stat", shell=True, stdout=subprocess.PIPE).communicate()[0].decode('utf-8')
    else:
        tax=int(tax)
        out = subprocess.Popen(f"esearch -db genome -query 'txid{tax}' |elink -target assembly|esummary| xtract -pattern DocumentSummary \
                -def NA -element Organism,AssemblyAccession,Taxid,assembly-status -block Stat \
                -if Stat@category -equals contig_count -or Stat@category -equals contig_l50 \
                -or Stat@category -equals contig_n50 -or Stat@category -equals total_length \
                -element Stat", shell=True, stdout=subprocess.PIPE).communicate()[0].decode('utf-8')

    raw_lst = []
    lst = out.strip().split("\n")
    for i in lst:
        raw_lst.append(i.split("\t"))
    df = pd.DataFrame(raw_lst)
    df.columns =['Organism', 'AssemblyAccession', 
                    'Taxid', 'assembly-status', 'contig_count',
                    'contig_l50', 'contig_n50', 'total_length']

    df["Organism"].unique()
    df = df.astype({"contig_count": int})
    return df


def filter_taxa_df(df):
    print("\033[;33;1mPlease Select an Organism: \033[;39;m")
    counter = 1
    uniques = df["Organism"].unique()
    for i in uniques:
        print(f"{counter}: {i}")
        counter += 1
    n = int(input("\n"))
    m = n-1
    out = df[df["Organism"] == df["Organism"].unique()[m]].sort_values("contig_count")
    
    print("\033[;33;1mPlease Select an Assembly level: \033[;39;m")
    counter = 1
    uniques = out["assembly-status"].unique()
    for i in uniques:
        print(f"{counter}: {i}")
        counter += 1
    n = int(input("\n"))
    m = n-1
    out2 = out[out["assembly-status"] == out["assembly-status"].unique()[m]].sort_values("contig_count")
    return out2

def download_ref(df):
    df2 = df[["AssemblyAccession", "total_length", "assembly-status", "contig_count", "contig_l50", "contig_n50"]]
    df2.index = df2.reset_index(drop = True).index + 1

    if len(df2.index) > 1:
        print("\033[;33;1mPlease Select a Number to Download: \033[;39;m")
        print(df2.to_string())
        n = int(input("\n"))
        m = n-1
        acc = df2.iloc[m]["AssemblyAccession"]
    else:
        print(df2.to_string())
        acc = df2.iloc[0]["AssemblyAccession"]

    os.system(f"./get_acc.sh {acc}")
    print("Done")


In [ ]:
df = pd.read_table("EL.report")
df.columns =['perc', 'clade', 'assigned', 'rank', 'txid', 'name']
out = df[df["rank"] == "S" ].sort_values("perc",ascending=False)
out.index = out.reset_index(drop = True).index + 1
name = str(out.iloc[0]["name"]).strip()

In [ ]:
df = retrive_ref(name)
df_filterred = filter_taxa_df(df)
download_ref(df_filterred)
